# Its Modeling Time Baby!!!!!!!!!!!!!!

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
%matplotlib inline
plt.style.use("ggplot")
import pylab
pylab.rcParams['figure.figsize'] = (15, 10)

from scipy import stats
!pip install statsmodels
import statsmodels.api as sm
import time

from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metric
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

     |████████████████████████████████| 9.5 MB 10.3 MB/s 
     |████████████████████████████████| 231 kB 75.9 MB/s 


In [ ]:
import datetime
datetime.datetime.now()

datetime.datetime(2021, 9, 23, 22, 30, 57, 987895)

In [ ]:
app_train = pd.read_csv("app_train_red.csv", index_col=0)
app_test = pd.read_csv("app_test_red.csv")

In [ ]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0,0,0,0,0,0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0,0,0,0,0,0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,0,0,0,0,0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
app_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0,0,0,0,0,0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0,0,0,0,0,0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0,0,0,0,0,0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0,0,0,0,0,0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
target = app_train.TARGET
train_data = app_train.drop(["SK_ID_CURR", "TARGET"], axis = 1)
test_data = app_test.drop("SK_ID_CURR", axis = 1)
train_data.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Working,...,0,0,0,0,0,0,0,0,0,0
1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,State servant,...,0,0,0,0,0,0,0,0,0,0
2,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Working,...,0,0,0,0,0,0,0,0,0,0
3,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Working,...,0,0,0,0,0,0,0,0,0,0
4,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Working,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
binary_cols = [col for col in train_data if col.startswith("FLAG")] + ["CODE_GENDER",  "REG_REGION_NOT_LIVE_REGION", "REG_REGION_NOT_WORK_REGION", 
                "LIVE_REGION_NOT_WORK_REGION", "REG_CITY_NOT_LIVE_CITY", "REG_CITY_NOT_WORK_CITY", "LIVE_CITY_NOT_WORK_CITY",]

cat_cols = ["NAME_CONTRACT_TYPE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", 
            "OCCUPATION_TYPE", "REGION_RATING_CLIENT", "REGION_RATING_CLIENT_W_CITY", "ORGANIZATION_TYPE"] 

num_cols = list(set(train_data.columns) - set(cat_cols) - set(binary_cols))
num_cols

['OBS_30_CNT_SOCIAL_CIRCLE',
 'DAYS_EMPLOYED',
 'OWN_CAR_AGE',
 'REGION_POPULATION_RELATIVE',
 'AMT_CREDIT',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'AMT_GOODS_PRICE',
 'EXT_SOURCE_2',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_3',
 'AMT_INCOME_TOTAL',
 'DAYS_BIRTH',
 'DAYS_LAST_PHONE_CHANGE',
 'AMT_ANNUITY',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'CNT_FAM_MEMBERS',
 'CNT_CHILDREN']

In [ ]:
train_data[["FLAG_OWN_CAR", "FLAG_OWN_REALTY"]] = train_data[["FLAG_OWN_CAR", "FLAG_OWN_REALTY"]].replace({"Y":1, "N":0}, )
train_data["CODE_GENDER"] = train_data["CODE_GENDER"].replace({"F":1, "M":0, "XNA":1})

In [ ]:
# Missing Values
missing_vals = pd.DataFrame(train_data.isnull().sum()).reset_index()
missing_vals.columns = ["Variable", "num_missing"]
missing_vals["percent_missing"] = missing_vals.num_missing / len(train_data)
missing_vals[missing_vals.num_missing > 0]

,Variable,num_missing,percent_missing
7,AMT_ANNUITY,12,0.000039
8,AMT_GOODS_PRICE,278,0.000904
18,OWN_CAR_AGE,202929,0.659908
25,OCCUPATION_TYPE,96391,0.313455
26,CNT_FAM_MEMBERS,2,0.000007
36,EXT_SOURCE_1,173378,0.563811
37,EXT_SOURCE_2,660,0.002146
38,EXT_SOURCE_3,60965,0.198253
39,OBS_30_CNT_SOCIAL_CIRCLE,1021,0.003320
40,DEF_30_CNT_SOCIAL_CIRCLE,1021,0.003320


## Preprocess Data

In [ ]:
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy = "median")),
    ('scaler', StandardScaler())
])


# cat_transformer = Pipeline([
#     ('imputer', SimpleImputer(strategy = "constant", fill_value = "missing")),
#     ('onehot', OneHotEncoder())
# ])

cat_transformer = OneHotEncoder()

data_pipe = ColumnTransformer([
    ("cat_transformer", cat_transformer, cat_cols),
    ('num_transformer', num_transformer, num_cols)
], remainder = "passthrough")

train_data_transf = pd.DataFrame(data_pipe.fit_transform(train_data))

In [ ]:
# data_pipe.named_transformers_["num_transformer"].get_feature_names(num_cols)
train_data_transf.columns = list(data_pipe.named_transformers_["cat_transformer"].get_feature_names(cat_cols)) + num_cols + binary_cols
train_data_transf.head()

In [ ]:
#Check Missing Values after data transformation and imputation
missing_vals = pd.DataFrame(train_data_transf.isnull().sum()).reset_index()
missing_vals.columns = ["Variable", "num_missing"]
missing_vals["percent_missing"] = missing_vals.num_missing / len(train_data_transf)
missing_vals[missing_vals.num_missing > 0]

,Variable,num_missing,percent_missing


## Baseline Model
- Model is based on just the data in the current application
- Minimal to no feature engineering
- Some irrelevant data was dropped

In [ ]:
start_time = time.time()
logit_model = LogisticRegression(random_state = 42, max_iter = 5000)

param_grid = [
    {"solver":["saga"], "penalty":["none"]}, 
    {"solver":["saga"], "penalty":["l1", "l2"], "C":[0.001, 0.01, 0.1, 1]}, 
    {"solver":["saga"], "penalty":["elasticnet"], "l1_ratio":[0.1, 0.3, 0.5, 0.7]}
]

rand_search = RandomizedSearchCV(logit_model, param_grid, random_state = 42, cv = 3, scoring = "neg_log_loss")
rand_search.fit(train_data_transf[num_cols], target)
print(rand_search.best_params_)
logit_base = rand_search.best_estimator_

print(datetime.datetime.now())

print('------- Time to execute: %f -------' %(time.time() - start_time))

{'solver': 'saga', 'penalty': 'none'}
2021-09-23 23:33:57.910092
------- Time to execute: 3773.498205 -------


In [ ]:
logit_base_pred = logit_base.predict(train_data_transf[num_cols])
# pd.DataFrame(logit_base_pred).to_csv("logit_base_preds.csv")

In [ ]:
metric.confusion_matrix(target, logit_base_pred)

array([[282526,    160],
       [ 24675,    150]])

In [ ]:
print("Accuracy Score: %f \n Recall Score: %f \n Precision Score %f" 
    % (metric.accuracy_score(target, logit_base_pred), metric.precision_score(target, logit_base_pred), metric.recall_score(target, logit_base_pred)))

Accuracy Score: 0.919239 
 Recall Score: 0.483871 
 Precision Score 0.006042


In [ ]:
round(metric.accuracy_score(target, preds), 4) * 100

NameError: name 'preds' is not defined

In [ ]:
logit_base

LogisticRegression(C=1, max_iter=5000, random_state=42, solver='newton-cg')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9263f292-ea73-48e7-8892-f3d3ed1db03c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>